In [1]:
import numpy as np
import matplotlib.pyplot as plt

from keras.datasets import cifar10
from keras.layers import Dense, Activation, Flatten
from keras.models import Sequential, Model, load_model
from keras.applications.densenet import DenseNet121, preprocess_input
from keras.utils import to_categorical
from keras import backend as K

Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [3]:
base = DenseNet121(include_top=False, pooling='avg', input_shape=(32, 32, 3))

x = Dense(10, name='fc')(base.output)
x = Activation('softmax', name='prediction')(x)

model = Model(inputs=base.input, outputs=x)

In [4]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 16, 16, 64)   9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 16, 16, 64)   256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

In [5]:
#print(len(model.trainable_weights))
for layer in model.layers:
    layer.trainable = False
model.layers[-1].trainable = True
model.layers[-2].trainable = True

print([layer.trainable for layer in model.layers])

[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False

In [14]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [15]:
history = model.fit(x_train, y_train, epochs=3, batch_size=32, validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/3
50000/50000 [==============================] - 41s 814us/step - loss: 1.3135 - categorical_accuracy: 0.5411 - val_loss: 1.8281 - val_categorical_accuracy: 0.4492
Epoch 2/3
50000/50000 [==============================] - 39s 780us/step - loss: 1.3119 - categorical_accuracy: 0.5410 - val_loss: 1.6085 - val_categorical_accuracy: 0.4900
Epoch 3/3
50000/50000 [==============================] - 39s 776us/step - loss: 1.3067 - categorical_accuracy: 0.5455 - val_loss: 1.7377 - val_categorical_accuracy: 0.4647


In [8]:
model.evaluate(x_test, y_test)

10000/10000 [==============================] - 3s 313us/step


[0.18599436459541321, 0.9276999905586243]

In [9]:
model.save('../tmp/cifar10_vgg16_pretrain_test.h5')

In [9]:
y_test[0]

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0.], dtype=float32)